In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

In [5]:
# 1. 데이터 확보
import zipfile
filepath = 'data/glaucoma.zip'
with zipfile.ZipFile(filepath, 'r') as f:
    f.extractall('data/')

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
                    rotation_range=20, # 최대 회전 각도
                    width_shift_range=0.2, # 가로 방향 -20%~20% 이동 정도
                    height_shift_range=0.2, # 상하 방향 이동 정도
                    horizontal_flip=True, # 수평 뒤집기
                    vertical_flip=True) # 수직 뒤집기
train_data = train_generator.flow_from_directory(
        'data/glaucoma/train',
        target_size=(224,224),
        batch_size=4,
        class_mode='sparse' # 종속변수에 원핫인코딩 안 함
)
test_generator = ImageDataGenerator()
test_data = test_generator.flow_from_directory(
    'data/glaucoma/test',
    target_size=(224,224),
    batch_size=4,
    class_mode='sparse'
)

Found 828 images belonging to 3 classes.
Found 320 images belonging to 3 classes.


In [9]:
# 학습데이터와 테스트 데이터가 될 이미지의 채널 확인
img = image.load_img('data/glaucoma/test/normal_control/1.png')
img_data = image.img_to_array(img)
img_data.shape

(240, 240, 3)

In [8]:
# 2. 모델 구성
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np
from tensorflow.keras.models import Sequential # ResNet50의 Dense 층을 사용자 추가
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing import image # 이미지를 읽어오기

resnet_model = ResNet50(input_shape=(224,224,3), 
                        include_top=False) # 컨볼루션과 폴링 레이어만 